In [1]:
from transformers import Trainer, TrainingArguments, BertForSequenceClassification, BertTokenizer
from datasets import Dataset
import pandas as pd
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

# Load the WELFake dataset
df = pd.read_csv('WELFake_Dataset.csv')
df = df.dropna(subset=['text'])

# Convert the 'text' column to string type
df['text'] = df['text'].astype(str)

# Convert the pandas DataFrame to HuggingFace Dataset
dataset = Dataset.from_pandas(df)


# Split the dataset into training and testing sets (80% training, 20% testing)
train_test_split = dataset.train_test_split(test_size=0.2)
train_dataset = train_test_split['train']
test_dataset = train_test_split['test']

# Define a function to compute evaluation metrics: precision, recall, F1 score, and accuracy
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    
    # Compute accuracy
    acc = accuracy_score(labels, preds)
    
    # Compute precision, recall, and F1 score
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')

    return {
        'accuracy': round(acc, 5),
        'precision': round(precision, 5),
        'recall': round(recall, 5),
        'f1': round(f1, 5),
    }

# List of BERT-based tokenizers and models
models = [
    ("bert-base-uncased", "bert-base-uncased"),
    ("bert-base-cased", "bert-base-cased"),
    ("distilbert-base-uncased", "distilbert-base-uncased"),
    ("bert-base-multilingual-cased", "bert-base-multilingual-cased"),
    ("albert-base-v2", "albert-base-v2"),
    ("Roberta-base", "roberta-base"),
    ("allenai/scibert_scivocab_uncased", "allenai/scibert_scivocab_uncased")
]

# Training arguments (same for all models)
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    num_train_epochs=3,
    weight_decay=0.01,
    save_strategy="epoch",  # Save model at each epoch
)

def train_model(model_name, tokenizer_name):
    print(f"Training and evaluating {model_name}...")

    # Initialize tokenizer and model
    tokenizer = BertTokenizer.from_pretrained(tokenizer_name)
    model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)

    # Tokenize the dataset
    def tokenize_function(examples):
        return tokenizer(examples['text'], padding='max_length', truncation=True, max_length=512)

    tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
    tokenized_test_dataset = test_dataset.map(tokenize_function, batched=True)

    # Initialize the Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_train_dataset,
        eval_dataset=tokenized_test_dataset,
        compute_metrics=compute_metrics
    )

    # Train the model
    trainer.train()

    # Evaluate the model
    eval_results = trainer.evaluate()

    # Print evaluation results
    print(f"Results for {model_name}: {eval_results}")

    # Save the model
    model.save_pretrained(f'./{model_name}_trained_model')
    tokenizer.save_pretrained(f'./{model_name}_trained_model')

    print(f"Model {model_name} trained and saved successfully!")

# Now you can manually select a model to train
# Example: Train BERT-base-uncased
#train_model("bert-base-uncased", "bert-base-uncased")

# To train a different model, just call train_model with another pair from the models list
# Example: Train BERT-base-cased
# train_model("bert-base-cased", "bert-base-cased")


C:\Users\kprompol\AppData\Local\anaconda3\Lib\site-packages\transformers\training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [2]:
train_model("bert-base-cased", "bert-base-cased")

Training and evaluating bert-base-cased...


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

C:\Users\kprompol\AppData\Local\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:147: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\kprompol\.cache\huggingface\hub\models--bert-base-cased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

...

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/57676 [00:00<?, ? examples/s]

Map:   0%|          | 0/14419 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.040100,0.025346,0.993690,0.994440,0.993230,0.993830
2,0.015700,0.036932,0.993620,0.997950,0.989570,0.993740
3,0.001600,0.028043,0.995210,0.994990,0.995670,0.995330


Results for bert-base-cased: {'eval_loss': 0.028043357655405998, 'eval_accuracy': 0.99521, 'eval_precision': 0.99499, 'eval_recall': 0.99567, 'eval_f1': 0.99533, 'eval_runtime': 109.068, 'eval_samples_per_second': 132.202, 'eval_steps_per_second': 2.072, 'epoch': 3.0}
Model bert-base-cased trained and saved successfully!


In [3]:
train_model("distilbert-base-uncased", "distilbert-base-uncased")

Training and evaluating distilbert-base-uncased...


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

C:\Users\kprompol\AppData\Local\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:147: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\kprompol\.cache\huggingface\hub\models--distilbert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

...

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'embeddings.LayerNorm.bias', 'embeddings.LayerNorm.weight', 'embeddings.position_embeddings.weight', 'embeddings.token_type_embeddings.weight', 'embeddings.word_embeddings.weight', 'encoder.layer.0.attention.output.LayerNorm.bias', 'encoder.layer.0.attention.output.LayerNorm.weight', 'encoder.layer.0.attention.output.dense.bias', 'encoder.layer.0.attention.output.dense.weight', 'encoder.layer.0.attention.self.key.bias', 'encoder.layer.0.attention.self.key.weight', 'encoder.layer.0.attention.self.query.bias', 'encoder.layer.0.attention.self.query.weight', 'encoder.layer.0.attention.self.value.bias', 'encoder.layer.0.attention.self.value.weight', 'encoder.layer.0.intermediate.dense.bias', 'encoder.layer.0.intermediate.dense.weight', 'encoder.layer.0.output.LayerNorm.bias', 'encoder.layer.0.output.LayerN

Map:   0%|          | 0/57676 [00:00<?, ? examples/s]

Map:   0%|          | 0/14419 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.085800,0.081028,0.982520,0.978010,0.988080,0.983020
2,0.051800,0.060492,0.987100,0.987670,0.987130,0.987400
3,0.024200,0.066742,0.987590,0.983880,0.992010,0.987930


Results for distilbert-base-uncased: {'eval_loss': 0.06674216687679291, 'eval_accuracy': 0.98759, 'eval_precision': 0.98388, 'eval_recall': 0.99201, 'eval_f1': 0.98793, 'eval_runtime': 106.2916, 'eval_samples_per_second': 135.655, 'eval_steps_per_second': 2.126, 'epoch': 3.0}
Model distilbert-base-uncased trained and saved successfully!


In [2]:
train_model("bert-base-multilingual-cased", "bert-base-multilingual-cased")

Training and evaluating bert-base-multilingual-cased...


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

C:\Users\kprompol\AppData\Local\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:147: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\kprompol\.cache\huggingface\hub\models--bert-base-multilingual-cased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

...

Map:   0%|          | 0/57676 [00:00<?, ? examples/s]

Map:   0%|          | 0/14419 [00:00<?, ? examples/s]

C:\Users\kprompol\AppData\Local\anaconda3\Lib\site-packages\transformers\models\bert\modeling_bert.py:439: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.043300,0.039857,0.989460,0.992120,0.987290,0.989690
2,0.022200,0.030500,0.992650,0.989920,0.995810,0.992850
3,0.006900,0.029693,0.994310,0.995660,0.993240,0.994450


Results for bert-base-multilingual-cased: {'eval_loss': 0.029692603275179863, 'eval_accuracy': 0.99431, 'eval_precision': 0.99566, 'eval_recall': 0.99324, 'eval_f1': 0.99445, 'eval_runtime': 107.0798, 'eval_samples_per_second': 134.657, 'eval_steps_per_second': 2.111, 'epoch': 3.0}
Model bert-base-multilingual-cased trained and saved successfully!


In [1]:
from transformers import Trainer, TrainingArguments, AlbertTokenizer, AlbertForSequenceClassification
from datasets import Dataset
import pandas as pd
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

# Load the WELFake dataset
# Load the WELFake dataset
df = pd.read_csv('WELFake_Dataset.csv')
df = df.dropna(subset=['text'])

# Convert the 'text' column to string type
df['text'] = df['text'].astype(str)
dataset = Dataset.from_pandas(df)

# Split the dataset into training and testing sets (80% training, 20% testing)
train_test_split = dataset.train_test_split(test_size=0.2)
train_dataset = train_test_split['train']
test_dataset = train_test_split['test']

# Define a function to compute evaluation metrics: precision, recall, F1 score, and accuracy
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    
    # Compute accuracy
    acc = accuracy_score(labels, preds)
    
    # Compute precision, recall, and F1 score
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')

    return {
        'accuracy': round(acc, 5),
        'precision': round(precision, 5),
        'recall': round(recall, 5),
        'f1': round(f1, 5),
    }

# Training arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    num_train_epochs=3,
    weight_decay=0.01,
    save_strategy="epoch",  # Save model at each epoch
)

def train_model_albert():
    print(f"Training and evaluating ALBERT-base-v2...")

    # Initialize ALBERT tokenizer and model
    tokenizer = AlbertTokenizer.from_pretrained('albert-base-v2')
    model = AlbertForSequenceClassification.from_pretrained('albert-base-v2', num_labels=2)

    # Tokenize the dataset
    def tokenize_function(examples):
        return tokenizer(examples['text'], padding='max_length', truncation=True, max_length=512)

    tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
    tokenized_test_dataset = test_dataset.map(tokenize_function, batched=True)

    # Initialize the Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_train_dataset,
        eval_dataset=tokenized_test_dataset,
        compute_metrics=compute_metrics
    )

    # Train the model
    trainer.train()

    # Evaluate the model
    eval_results = trainer.evaluate()

    # Print evaluation results
    print(f"Results for ALBERT-base-v2: {eval_results}")

    # Save the model
    model.save_pretrained(f'./albert-base-v2_trained_model')
    tokenizer.save_pretrained(f'./albert-base-v2_trained_model')

    print(f"ALBERT-base-v2 model trained and saved successfully!")

# Train ALBERT-base-v2
train_model_albert()


C:\Users\kprompol\AppData\Local\anaconda3\Lib\site-packages\transformers\training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Training and evaluating ALBERT-base-v2...


spiece.model:   0%|          | 0.00/760k [00:00<?, ?B/s]

C:\Users\kprompol\AppData\Local\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:147: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\kprompol\.cache\huggingface\hub\models--albert-base-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
C:\Users\kprompol\AppData\Local\anaconda3\Lib\site-packages\transformers\tokenization_utils_base.py:1601: Futu

model.safetensors:   0%|          | 0.00/47.4M [00:00<?, ?B/s]

Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at albert-base-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/57676 [00:00<?, ? examples/s]

Map:   0%|          | 0/14419 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.048900,0.044952,0.991820,0.989300,0.994890,0.992080
2,0.028000,0.043127,0.991820,0.986040,0.998250,0.992110
3,0.012600,0.016859,0.996670,0.996640,0.996910,0.996770


Results for ALBERT-base-v2: {'eval_loss': 0.016858626157045364, 'eval_accuracy': 0.99667, 'eval_precision': 0.99664, 'eval_recall': 0.99691, 'eval_f1': 0.99677, 'eval_runtime': 132.9295, 'eval_samples_per_second': 108.471, 'eval_steps_per_second': 1.7, 'epoch': 3.0}
ALBERT-base-v2 model trained and saved successfully!


In [7]:
from transformers import Trainer, TrainingArguments, RobertaTokenizer, RobertaForSequenceClassification
from datasets import Dataset
import pandas as pd
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

# Load the WELFake dataset
# Load the WELFake dataset
df = pd.read_csv('WELFake_Dataset.csv')
df = df.dropna(subset=['text'])

# Convert the 'text' column to string type
df['text'] = df['text'].astype(str)
dataset = Dataset.from_pandas(df)

# Split the dataset into training and testing sets (80% training, 20% testing)
train_test_split = dataset.train_test_split(test_size=0.2)
train_dataset = train_test_split['train']
test_dataset = train_test_split['test']

# Define a function to compute evaluation metrics: precision, recall, F1 score, and accuracy
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    
    # Compute accuracy
    acc = accuracy_score(labels, preds)
    
    # Compute precision, recall, and F1 score
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')

    return {
        'accuracy': round(acc, 5),
        'precision': round(precision, 5),
        'recall': round(recall, 5),
        'f1': round(f1, 5),
    }

# Training arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    num_train_epochs=3,
    weight_decay=0.01,
    save_strategy="epoch",  # Save model at each epoch
)

def train_model_roberta():
    print(f"Training and evaluating RoBERTa-base...")

    # Initialize RoBERTa tokenizer and model
    tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
    model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=2)

    # Tokenize the dataset
    def tokenize_function(examples):
        return tokenizer(examples['text'], padding='max_length', truncation=True, max_length=512)

    tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
    tokenized_test_dataset = test_dataset.map(tokenize_function, batched=True)

    # Initialize the Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_train_dataset,
        eval_dataset=tokenized_test_dataset,
        compute_metrics=compute_metrics
    )

    # Train the model
    trainer.train()

    # Evaluate the model
    eval_results = trainer.evaluate()

    # Print evaluation results
    print(f"Results for RoBERTa-base: {eval_results}")

    # Save the model
    model.save_pretrained(f'./roberta-base_trained_model')
    tokenizer.save_pretrained(f'./roberta-base_trained_model')

    print(f"RoBERTa-base model trained and saved successfully!")

# Train RoBERTa-base
train_model_roberta()


C:\Users\kprompol\AppData\Local\anaconda3\Lib\site-packages\transformers\training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Training and evaluating RoBERTa-base...


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

C:\Users\kprompol\AppData\Local\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:147: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\kprompol\.cache\huggingface\hub\models--roberta-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/57676 [00:00<?, ? examples/s]

Map:   0%|          | 0/14419 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.010200,0.011588,0.998130,0.998630,0.997670,0.998150
2,0.005600,0.010838,0.998610,0.999730,0.997530,0.998630
3,0.002000,0.006605,0.999170,0.999590,0.998770,0.999180


Results for RoBERTa-base: {'eval_loss': 0.0066045126877725124, 'eval_accuracy': 0.99917, 'eval_precision': 0.99959, 'eval_recall': 0.99877, 'eval_f1': 0.99918, 'eval_runtime': 112.5402, 'eval_samples_per_second': 128.123, 'eval_steps_per_second': 2.008, 'epoch': 3.0}
RoBERTa-base model trained and saved successfully!


In [8]:
from transformers import Trainer, TrainingArguments, AutoTokenizer, AutoModelForSequenceClassification
from datasets import Dataset
import pandas as pd
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
import torch

# Load the WELFake dataset
df = pd.read_csv('WELFake_Dataset.csv')
df = df.dropna(subset=['text'])

# Convert the 'text' column to string type
df['text'] = df['text'].astype(str)
dataset = Dataset.from_pandas(df)

# Split the dataset into training and testing sets (80% training, 20% testing)
train_test_split = dataset.train_test_split(test_size=0.2)
train_dataset = train_test_split['train']
test_dataset = train_test_split['test']

# Define a function to compute evaluation metrics: precision, recall, F1 score, and accuracy
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    
    # Compute accuracy
    acc = accuracy_score(labels, preds)
    
    # Compute precision, recall, and F1 score
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')

    return {
        'accuracy': round(acc, 5),
        'precision': round(precision, 5),
        'recall': round(recall, 5),
        'f1': round(f1, 5),
    }

# Training arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    num_train_epochs=3,
    weight_decay=0.01,
    save_strategy="epoch",  # Save model at each epoch
    save_safetensors=False,  # Disable safetensors to avoid the non-contiguous tensor issue
)

def train_model(model_name):
    print(f"Training and evaluating {model_name}...")

    # Initialize tokenizer and model
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

    # Tokenize the dataset
    def tokenize_function(examples):
        return tokenizer(examples['text'], padding='max_length', truncation=True, max_length=512)

    tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
    tokenized_test_dataset = test_dataset.map(tokenize_function, batched=True)

    # Initialize the Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_train_dataset,
        eval_dataset=tokenized_test_dataset,
        compute_metrics=compute_metrics
    )

    # Train the model
    trainer.train()

    # Evaluate the model
    eval_results = trainer.evaluate()

    # Print evaluation results
    print(f"Results for {model_name}: {eval_results}")

    # Save the model with standard serialization
    print(f"Saving model for {model_name}...")
    model.save_pretrained(f'./{model_name}_trained_model')
    tokenizer.save_pretrained(f'./{model_name}_trained_model')

    print(f"Model {model_name} trained and saved successfully!")

# Train SciBERT (or any other model using AutoTokenizer and AutoModel)
train_model("allenai/scibert_scivocab_uncased")


C:\Users\kprompol\AppData\Local\anaconda3\Lib\site-packages\transformers\training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Training and evaluating allenai/scibert_scivocab_uncased...


C:\Users\kprompol\AppData\Local\anaconda3\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at allenai/scibert_scivocab_uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/57676 [00:00<?, ? examples/s]

Map:   0%|          | 0/14419 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.044000,0.028085,0.993060,0.989250,0.997460,0.993340
2,0.019900,0.027943,0.994520,0.996240,0.993180,0.994710
3,0.005300,0.035532,0.994660,0.996640,0.993040,0.994840


Results for allenai/scibert_scivocab_uncased: {'eval_loss': 0.03553220257163048, 'eval_accuracy': 0.99466, 'eval_precision': 0.99664, 'eval_recall': 0.99304, 'eval_f1': 0.99484, 'eval_runtime': 101.7571, 'eval_samples_per_second': 141.7, 'eval_steps_per_second': 2.221, 'epoch': 3.0}
Saving model for allenai/scibert_scivocab_uncased...


ValueError: You are trying to save a non contiguous tensor: `bert.encoder.layer.0.attention.self.query.weight` which is not allowed. It either means you are trying to save tensors which are reference of each other in which case it's recommended to save only the full tensors, and reslice at load time, or simply call `.contiguous()` on your tensor to pack it before saving.

In [11]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Reinitialize the model and tokenizer (replace with your model if already trained and not saved yet)
model_name = "allenai/scibert_scivocab_uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

def save_model(model, model_name, tokenizer):
    # Ensure all tensors are contiguous before saving the model
    print(f"Making all tensors contiguous for {model_name}...")
    for param in model.parameters():
        param.data = param.data.contiguous()

    # Save the model
    print(f"Saving model for {model_name}...")
    model.save_pretrained(f'./{model_name}_trained_model')
    tokenizer.save_pretrained(f'./{model_name}_trained_model')

    print(f"Model {model_name} saved successfully!")


C:\Users\kprompol\AppData\Local\anaconda3\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at allenai/scibert_scivocab_uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
# Assume you have already trained the model and have access to the model and tokenizer
save_model(model, "allenai/scibert_scivocab_uncased", tokenizer)

Making all tensors contiguous for allenai/scibert_scivocab_uncased...
Saving model for allenai/scibert_scivocab_uncased...
Model allenai/scibert_scivocab_uncased saved successfully!
